In [4]:
import pandas as pd
import numpy as np
from sklearn import datasets
import matplotlib
%matplotlib inline

In [5]:
iris = datasets.load_iris()
df = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
i = pd.Series(iris['target'])
names = i.map(lambda x: iris['target_names'][x])
df['index'] = i
df['category_name'] = names
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),index,category_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
5,5.4,3.9,1.7,0.4,0,setosa
6,4.6,3.4,1.4,0.3,0,setosa
7,5.0,3.4,1.5,0.2,0,setosa
8,4.4,2.9,1.4,0.2,0,setosa
9,4.9,3.1,1.5,0.1,0,setosa


In [15]:
import edward as ed
import tensorflow as tf
from edward.models import Bernoulli, MultivariateNormalTriL, Normal
from edward.util import rbf

ys = df['index'].values
xs = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']].values
N = xs.shape[0]
D = xs.shape[1]
print("Number of data points: {}".format(N))
print("Number of features: {}".format(D))

Number of data points: 150
Number of features: 4


In [19]:
X = tf.placeholder(tf.float32, [N, D])
f = MultivariateNormalTriL(loc=tf.zeros(N), scale_tril=tf.cholesky(rbf(X)))
y = Bernoulli(logits=f)

In [20]:
qf = Normal(loc=tf.get_variable("qf/loc", [N]),
            scale=tf.nn.softplus(tf.get_variable("qf/scale", [N])))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-b7a45d5af1e7>", line 1, in <module>
    qf = Normal(loc=tf.get_variable("qf/loc", [N]),
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/tensorflow/python/ops/variable_scope.py", line 1297, in get_variable
    constraint=constraint)
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/tensorflow/python/ops/variable_scope.py", line 1093, in get_variable
    constraint=constraint)
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/tensorflow/python/ops/variable_scope.py", line 439, in get_variable
    constraint=constraint)
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/tensorflow/python/ops/variable_scope.py", line 408, in _true_getter
    use_resource=us

ValueError: Variable qf/loc already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-11-b7a45d5af1e7>", line 1, in <module>
    qf = Normal(loc=tf.get_variable("qf/loc", [N]),
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2903, in run_ast_nodes
    if self.run_code(code, result):


In [21]:
inference = ed.KLqp({f: qf}, data={X: xs, y: ys})
inference.run(n_iter=5000)

/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1361, in _do_call
    return fn(*args)
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1340, in _run_fn
    target_list, status, run_metadata)
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-packages/tensorflow/python/framework/errors_impl.py", line 516, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.InvalidArgumentError: Cholesky decomposition was not successful. The input might not be valid.
	 [[Node: inference_2/sample_2/Cholesky_2 = Cholesky[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](inference_2/sample_2/mul_11)]]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/app-user/.pyenv/versions/3.6.3/lib/python3.6/site-

TypeError: must be str, not list